In [ ]:
import os
import pandas as pd
from konlpy.tag import Okt
from wordcloud import WordCloud
from tqdm import tqdm
from re import match
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
import pickle

In [ ]:
# 크롤링한 데이터가 있는 폴더
path = r"C:\0_Semi_project\data\sep_oct_2022"

# 폴더 내 파일 이름 가져오기
flst = os.listdir(path)
print(flst[:-1])
'''
['daum_news_20220901_20220915.pkl',
 'daum_news_20220916_20220930.pkl',
 'daum_news_20221001_20221014.pkl',
 'daum_news_20221015_20221031.pkl']
'''

In [ ]:
# 전체 데이터를 담을 빈 데이터프레임 만들기
news = pd.DataFrame()

# 피클 로드 & 데이터 합치기
for file in flst[:-1]:
    pk = pd.read_pickle(path + '/' + file)
    news = pd.concat([news, pk], ignore_index = True)

# 합친 전체 데이터 pickle 파일로 저장
news.to_pickle(path + '/daum_news_sep_oct_nov_2022.pkl')

In [ ]:
# 전체 뉴스 데이터 확인하기
news.info()
'''
RangeIndex: 97252 entries, 0 to 97251
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   title    97252 non-null  object
 1   url      97252 non-null  object
 2   press    97252 non-null  object
 3   content  97252 non-null  object
'''

In [ ]:
news.head(3)
'''
                                         title  ...                                            content
0      한문철도 놀란 자해공갈범.. 아파트 주차장서 후진 차량에 슬쩍 [영상]  ...    아파트 주차장에서 한 남성이 후진하는 차량에 고의로 부딪히고 운전자에게 피해보상...
1  임윤아, 현빈에 "정말 좋아했다고" 폭풍 오열..무슨 일? ('연중 라이브')  ...       [마이데일리 = 양유진 기자] 그룹 소녀시대 멤버 겸 배우 임윤아가 배우 ...
2                         뉴욕증시, 9월 첫 거래일 하락 출발  ...  (뉴욕=연합뉴스) 임하람 연합인포맥스 특파원 = 뉴욕증시는 9월의 첫 거래일을 하락...
'''

In [ ]:
okt = Okt()

# 명사 추출(기사)
content_nouns = []
for content in tqdm(news['content']):
    content_nouns.extend(okt.nouns(content))

In [ ]:
# 중간 백업
path = r'C:\0_Semi_project\data\seoul'

with open(path + '/daum_news_sep_oct_nov_nouns_2022.pkl', mode='wb') as f:
    pickle.dump(content_nouns, f)

In [ ]:
# 저장한 명사 가져오기

content_nouns = pd.read_pickle(path + '/' + '/seoul_nouns.pkl')
len(content_nouns) # 40246254

In [ ]:
# 단어 전처리

final_content_nouns = []

for noun in tqdm(content_nouns) :
    if len(noun) > 1 and not(match('^[0-9]', noun)): # 2음절 이상의 단어 선정, 숫자 제외
        final_content_nouns.append(noun)

len(final_content_nouns)

In [ ]:
# 뉴스 불용어 사전 (최종)
lib_path = r'C:\0_Semi_project\data\stopword'

with open(lib_path + "/뉴스불용어사전(최종)_231016.txt", "r", encoding="utf-8-sig") as file:
    lines = file.readlines()
    korean_lib = list(map(lambda x: x.replace("\n", ""), lines))

stopwords = list(set(korean_lib))

content_nouns=[]
for word in tqdm(final_content_nouns):
    if word not in stopwords:
        content_nouns.append(word)

len(content_nouns)

# 중간 백업
with open(path + '/2022_sep_oct_nov_nouns_process.pkl', mode='wb') as f:
    pickle.dump(content_nouns, f)

In [ ]:
word_count = Counter(content_nouns) # (dict, list)

In [ ]:
top500_word = word_count.most_common(500)

labels, values = zip(*top500_word)